In [ ]:
import pywatershed
import pandas as pd
import pathlib as pl
import numpy as np
import sys
sys.path.append('../scripts/')
from pest_utils import pars_to_tpl_entries
sys.path.append('../dependencies/')
import pyemu
interrupt_notebook = False

In [ ]:
all_models = ['01473000','05431486','09112500','14015000']# Create a list of all cutouts

In [ ]:
rootdir = pl.Path('../NHM_extractions/20230110_pois_haj/')# Path to location of cutouts

In [ ]:
cm = all_models[1] # sets cutout from list
pestdir = rootdir/ cm # stes path to location of NHM output folder where output files are.

In [ ]:
pst = pyemu.Pst(str(pestdir / 'prior_mc_loc.pst'))

In [ ]:
pst.plot(kind='phi_pie')

In [ ]:
pst.phi_components

#### read in the zero-weighting information

In [ ]:
zw_df = pd.read_csv('../Supporting_information/zero_weighting.csv', dtype={'cutout':str,'location':str})

In [ ]:
if isinstance(zw_df,pd.Series):
    zw_df = zw_df.to_frame().T
zw_df

In [ ]:
zw_df = zw_df.loc[zw_df.cutout==cm]
zw_df

### need to work in a copy of the observations dataframe to set various values properly to weight of 0

In [ ]:
obs_sf = pst.observation_data.loc[pst.observation_data.obgnme.str.contains('streamflow')].copy()


In [ ]:
obs_sf['month'] = 'mt'
obs_sf['location'] = [i.split(':')[-1] for i in obs_sf.index]
daily_index = obs_sf.loc[obs_sf.index.str.contains('daily'), 'obsnme'].to_list()
mean_mon_index = obs_sf.loc[obs_sf.index.str.contains('mean_mon'), 'obsnme'].to_list()
mon_index = obs_sf.loc[obs_sf.index.str.contains('streamflow_mon'), 'obsnme'].to_list()

In [ ]:
obs_sf.loc[daily_index, 'month'] = [i.split(':')[-2].split('_')[1] for i in daily_index]
obs_sf.loc[mean_mon_index, 'month'] = [i.split(':')[-2] for i in mean_mon_index]
obs_sf.loc[mon_index, 'month'] = [i.split(':')[-2].split('_')[1] for i in mon_index]


In [ ]:
for _,i in zw_df.iterrows():
    print(i.location, i.stmonth, i.endmonth)
    monthrange = [str(i) for i in range(i.stmonth, i.endmonth+1)]
    for cmonth in monthrange:
        obs_sf.loc[(obs_sf.location==i.location) & (obs_sf.month==cmonth),'weight'] = 0

In [ ]:
pst.observation_data.loc[obs_sf.index,'weight'] = obs_sf.weight

### now get on with reweighting

In [ ]:
phi = pst.phi
phi

In [ ]:
#stop

In [ ]:
# Assign relative contributions to the objective function
phi_new_comps = {'l_max_actet_mean_mon':0.04 * phi,
                 'g_min_actet_mean_mon':0.04 * phi,
                 'l_max_actet_mon':  .02 * phi,
                 'g_min_actet_mon':  .02 * phi,
                 'streamflow_nodata': 0 * phi,
                 'l_max_recharge_ann': 0.04 * phi,
                 'g_min_recharge_ann': 0.04 * phi,
                 'l_max_runoff_mon': .07 * phi,
                 'g_min_runoff_mon': .07 * phi,
                 'l_max_sca_daily':.05 * phi,
                 'g_min_sca_daily':.05 * phi,
                 'l_max_soil_moist_ann': 0.04 * phi,
                 'g_min_soil_moist_ann': 0.04 * phi,
                 'l_max_soil_moist_mon': 0.04 * phi,
                 'g_min_soil_moist_mon': 0.04 * phi,
                 'streamflow_mean_mon': .1 * phi,
                 'streamflow_mon': .1 * phi,
                }

In [ ]:
# add streamflow_daily components

#Changed above to '0' for now, rebalance above accordingly
obstmp = pst.observation_data.copy()
stream_comps_perc = 0.2 
low_comp = 0.6 * stream_comps_perc
high_comp = 0.4 * stream_comps_perc
low_stream_comps = high_stream_comps = 0
low_count = high_count = 0
for k,i in pst.phi_components.items():
    if k.lower().startswith('streamflow_daily'):
        if (('exlow' in k.lower()) | ('daily_low' in k.lower())):
            low_count += len(obstmp.loc[obstmp.obgnme==k])
            low_stream_comps += i
        else:
            high_count += len(obstmp.loc[obstmp.obgnme==k])
            high_stream_comps += i

for i, comp in pst.phi_components.items():
    if i.lower().startswith('streamflow_daily'):
        if (('exlow' in i.lower()) | ('daily_low' in i.lower())):
            phi_new_comps[i] = comp/low_stream_comps * low_comp * phi
        else:
            phi_new_comps[i] = comp/high_stream_comps * high_comp * phi


In [ ]:

low_stream_comps,high_stream_comps


In [ ]:
pst.phi_components.items()

In [ ]:
try:
    assert np.isclose(sum([i for _,i in phi_new_comps.items()])/phi, 1.0)
    print('Very very close to 1.0. Carry on!')
except:
    print('Not close to 1.0. Stop and recheck your weight for catagory.')

In [ ]:
sum([i for _,i in phi_new_comps.items()])/phi

In [ ]:
assert np.isclose(sum([i for _,i in phi_new_comps.items()])/phi, 1.0)

In [ ]:
pst.obs_groups

In [ ]:
pst.adjust_weights(obsgrp_dict=phi_new_comps)

In [ ]:
pst.plot(kind='phi_pie')

In [ ]:
pst.observation_data.loc[pst.observation_data.obgnme=='streamflow_nodata']

In [ ]:
#Write a new version of the PEST++ control file (.pst)
pst.control_data.noptmax=-1
pst.write(str(rootdir/cm/ 'prior_mc_reweight.pst'), version=2)


In [ ]:
print(pst.adjust_weights(obsgrp_dict=phi_new_comps))


In [ ]:
#Write this to a dataframe (.csv) and save it somewhere so that we have a record of it. with time date and append to .csv every notebook run.
print(pst.phi_components_normalized)


### update the localization matrix to remove groups with only 0-weighted obs

In [ ]:
# find zero-weighted groups
zero_grps = list(set(pst.obs_groups) - set(pst.nnz_obs_groups))
zero_grps

In [ ]:
# read in the localization matrix from the run directory
locmat = pyemu.Matrix.from_ascii(str(rootdir / cm / 'loc.mat')).to_dataframe()

In [ ]:
# confirm that we can select only the rows that are not in the zero-weighted groups lines
locmat.loc[~locmat.index.isin(zero_grps)]

In [ ]:
len(locmat),len(locmat.loc[~locmat.index.isin(zero_grps)])

In [ ]:
# write out the new matrix in PEST style
pyemu.Matrix.from_dataframe(locmat.loc[~locmat.index.isin(zero_grps)]).to_ascii(str(rootdir/cm/ 'loc.mat'))